In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [4]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Activation
from tensorflow.python.keras.optimizers import adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [5]:
Mucuri = pd.read_csv('./train150_mucuri.txt', sep='\t', header = None)

In [6]:
Mucuri.head(10)

,0,1,2,3,4,5,6,7,8
0,30,11,2015,14,13.012139,75.105481,27.516129,72.930636,1020.422601
1,30,11,2015,15,12.726087,68.334332,27.238095,75.212121,1020.394348
2,30,11,2015,16,12.081111,64.457865,27.105263,75.741379,1020.508333
3,30,11,2015,17,11.647222,53.842100,26.305556,75.302632,1020.611000
4,30,11,2015,18,11.064444,53.945279,25.464286,76.592593,1020.866500
5,30,11,2015,19,10.324444,51.320714,24.764706,83.344444,1021.286500
6,30,11,2015,20,9.862778,47.354507,24.310811,88.044643,1021.627500
7,30,11,2015,21,9.598889,41.816108,24.304348,91.329670,1021.892333
8,30,11,2015,22,9.488889,39.561270,24.500000,92.415385,1021.856167
9,30,11,2015,23,7.502222,37.881537,24.419355,93.629630,1021.834667


In [7]:
Mucuri.describe()

,0,1,2,3,4,5,6,7,8
count,550.000000,550.000000,550.0,550.000000,550.000000,550.000000,549.000000,550.000000,550.000000
mean,12.087273,11.981818,2015.0,11.496364,8.176719,73.027647,24.443445,85.263578,1017.313986
std,6.891273,0.133730,0.0,6.940745,2.724988,64.996263,1.669638,8.579562,8.796185
min,1.000000,11.000000,2015.0,0.000000,1.211905,1.448179,20.616279,46.355556,937.534333
25%,6.000000,12.000000,2015.0,5.250000,6.215278,38.605367,23.314815,80.942222,1016.544542
50%,12.000000,12.000000,2015.0,11.000000,8.320000,59.019307,24.336134,86.765805,1020.046750
75%,18.000000,12.000000,2015.0,17.750000,10.146806,83.329498,25.359375,91.434531,1021.497917
max,30.000000,12.000000,2015.0,23.000000,13.572222,356.553715,30.140625,98.000000,1023.726500


In [8]:
Mucuri.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
8    0
dtype: int64

In [9]:
Mucuri.update(Mucuri[6].fillna(Mucuri[6].mean()))

In [10]:
Mucuri.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64

In [11]:
x_array = Mucuri.values
x_array

array([[  30.        ,   11.        , 2015.        , ...,   27.51612903,
          72.93063584, 1020.422601  ],
       [  30.        ,   11.        , 2015.        , ...,   27.23809524,
          75.21212121, 1020.394348  ],
       [  30.        ,   11.        , 2015.        , ...,   27.10526316,
          75.74137931, 1020.508333  ],
       ...,
       [  23.        ,   12.        , 2015.        , ...,   22.60240964,
          79.80120482, 1016.323667  ],
       [  23.        ,   12.        , 2015.        , ...,   22.20325203,
          80.90740741, 1016.0715    ],
       [  23.        ,   12.        , 2015.        , ...,   22.02409639,
          84.76111111, 1014.882833  ]])

In [12]:
y_array = list(Mucuri[4].values)
y_array

[13.01213875,
 12.72608696,
 12.08111113,
 11.64722224,
 11.06444444,
 10.32444445,
 9.862777788999999,
 9.598888869,
 9.488888884,
 7.502222217000001,
 7.081111103,
 8.003333415,
 8.149444466,
 8.962777792,
 7.672777804,
 6.331666684,
 4.587179497,
 4.961111107,
 5.65944445,
 5.417777794,
 7.3039106039999995,
 8.602777796,
 11.70611108,
 12.30500005,
 12.08722222,
 11.34166668,
 10.69388894,
 9.519444438999999,
 8.94000002,
 8.105000016,
 7.637777778999999,
 6.661111122,
 4.801111111,
 3.717777774,
 2.141666657,
 2.77055556,
 2.7494444380000003,
 3.912222219,
 5.532222231,
 6.351666662,
 4.5777777760000005,
 3.918888871,
 4.281111113,
 3.5999999910000002,
 3.9055555539999998,
 7.156111113,
 8.167222222000001,
 8.547777798,
 9.712222205,
 10.48055558,
 10.28333334,
 9.587777767999999,
 9.037777797,
 7.906666687,
 7.552222284,
 6.913333315,
 6.856666679,
 6.588333339,
 6.380555564,
 6.203888903999999,
 6.588888878,
 7.843888892000001,
 8.235555543,
 8.783333349,
 6.128333334,
 3.1905555

In [13]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(9, activation='relu', input_shape=(9,)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear'),
  ])

  model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae'])
  return model

In [14]:
model = get_compiled_model()
model.fit(x_array, y_array, epochs=6)

W0823 10:16:07.963696 33572 deprecation.py:506] From D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/6
550/550 [==============================] - 0s 312us/sample - loss: 21186.6891 - mean_absolute_error: 127.2026
Epoch 2/6
550/550 [==============================] - 0s 57us/sample - loss: 744.6308 - mean_absolute_error: 24.0431
Epoch 3/6
550/550 [==============================] - 0s 43us/sample - loss: 498.1738 - mean_absolute_error: 20.2065
Epoch 4/6
550/550 [==============================] - 0s 48us/sample - loss: 106.7182 - mean_absolute_error: 6.5695
Epoch 5/6
550/550 [==============================] - 0s 46us/sample - loss: 62.6216 - mean_absolute_error: 5.3736
Epoch 6/6
550/550 [==============================] - 0s 50us/sample - loss: 32.4440 - mean_absolute_error: 4.2186


In [15]:
model.evaluate(x_array, y_array)

550/550 [==============================] - 0s 75us/sample - loss: 23.7275 - mean_absolute_error: 3.5333


[23.727543057528408, 3.5332701]

In [16]:
z_array = model.predict(x_array) 
z_array

array([[  8.361119  ],
       [  8.0817    ],
       [  7.9231925 ],
       [  7.616369  ],
       [  7.0815177 ],
       [  6.1809745 ],
       [  5.556127  ],
       [  5.2370353 ],
       [  5.515508  ],
       [  5.6194816 ],
       [ 10.966741  ],
       [ 10.652806  ],
       [ 10.437505  ],
       [ 10.785283  ],
       [ 10.927922  ],
       [ 11.22199   ],
       [ 11.27274   ],
       [ 12.167485  ],
       [ 13.768651  ],
       [ 12.082555  ],
       [ 12.914006  ],
       [ 10.56516   ],
       [ 11.19608   ],
       [ 11.346013  ],
       [ 11.104711  ],
       [ 11.205754  ],
       [ 10.895482  ],
       [ 10.689153  ],
       [ 10.596257  ],
       [ 10.41248   ],
       [ 10.216466  ],
       [ 10.211736  ],
       [ 10.101048  ],
       [ 10.096105  ],
       [ 10.405248  ],
       [ 12.184545  ],
       [ -8.539729  ],
       [ -7.5198007 ],
       [ -6.3080697 ],
       [ -9.155971  ],
       [ -6.410395  ],
       [ -5.41894   ],
       [ 17.04035   ],
       [ 17

In [17]:
z_array = model.predict(np.array([[30,11,2015,15,10.362423,75.10548099,27.51612903,72.93063584,1020.422601]])) 
print(z_array[0][0])

8.225469
